# 🇯🇵→🇰🇷 MarianMT 学習ノートブック

日本語→韓国語翻訳モデルの学習パイプライン

## 手順
1. 環境セットアップ
2. 教師データ生成 (NLLB-200)
3. MarianMT学習
4. ONNX変換・エクスポート

In [1]:
import os

## 0. GPU確認

In [2]:
!nvidia-smi

Fri Feb  6 06:21:54 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.127.05             Driver Version: 550.127.05     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4090        On  |   00000000:A1:00.0 Off |                  Off |
|  0%   28C    P8             22W /  450W |     728MiB /  24564MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## 1. 環境セットアップ

In [12]:
# RunPodの永続ディレクトリを設定
base_path = "/workspace/mt-ja-ko/mt-ja-ko"
os.makedirs(base_path, exist_ok=True)
%cd {base_path}

/workspace/mt-ja-ko/mt-ja-ko


In [4]:
!mkdir -p /workspace/huggingface_cache
!ls /workspace/huggingface_cache

evaluate  metrics  modules


In [5]:
# キャッシュ先を容量の大きい /workspace 配下に変更
os.environ['HF_HOME'] = '/workspace/huggingface_cache'
os.environ['HUGGINGFACE_HUB_CACHE'] = '/workspace/huggingface_cache'

In [6]:
rm -rf /root/.cache/huggingface/*

In [7]:
!df -h

Filesystem                   Size  Used Avail Use% Mounted on
overlay                       20G   16M   20G   1% /
tmpfs                         64M     0   64M   0% /dev
mfs#us-tx-3.runpod.net:9421  364T  153T  211T  43% /workspace
shm                           29G     0   29G   0% /dev/shm
/dev/md127                    14T  6.3T  7.8T  45% /etc/hosts
/dev/nvme0n1p2               1.8T   21G  1.7T   2% /usr/bin/nvidia-smi
tmpfs                        252G     0  252G   0% /sys/fs/cgroup
tmpfs                        252G   12K  252G   1% /proc/driver/nvidia
tmpfs                        252G  4.0K  252G   1% /etc/nvidia/nvidia-application-profiles-rc.d
tmpfs                         51G   34M   51G   1% /run/nvidia-persistenced/socket
tmpfs                        252G     0  252G   0% /proc/asound
tmpfs                        252G     0  252G   0% /proc/acpi
tmpfs                        252G     0  252G   0% /proc/scsi
tmpfs                        252G     0  252G   0% /sys/firmware
tmpfs

In [8]:
!pwd

/workspace/mt-ja-ko


In [3]:
# リポジトリのクローン
# すでにフォルダが存在する場合は pull して最新にします
repo_url = "https://github.com/nakaikento/mt-ja-ko.git"
repo_dir = "mt-ja-ko"

if not os.path.exists(repo_dir):
    print(f"Cloning repository from {repo_url}...")
    !git clone {repo_url}
    %cd {repo_dir}
else:
    print("Repository already exists. Pulling latest changes...")
    %cd {repo_dir}
    !git pull

Repository already exists. Pulling latest changes...
/workspace/mt-ja-ko
fatal: not a git repository (or any parent up to mount point /)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).


/usr/local/lib/python3.11/dist-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [6]:
!echo "typing_extensions>=4.10.0" >> requirements.txt

In [7]:
# 依存関係インストール
!pip install -r requirements.txt


[notice] A new release of pip is available: 24.2 -> 26.0.1
[notice] To update, run: python -m pip install --upgrade pip


## 2. データ確認

In [8]:
# データファイル確認
!ls -la data/splits/
!wc -l data/splits/*.ja data/splits/*.ko

ls: cannot access 'data/splits/': No such file or directory
wc: 'data/splits/*.ja': No such file or directory
wc: 'data/splits/*.ko': No such file or directory
0 total


In [11]:
# トークナイザー確認
!ls -la data/tokenized/

total 5338
drwxrwxrwx 2 root root 2000137 Feb  6 01:45 .
drwxrwxrwx 6 root root 2024298 Feb  6 01:45 ..
-rw-rw-rw- 1 root root  825905 Feb  6 01:45 spm.model
-rw-rw-rw- 1 root root  614203 Feb  6 01:45 spm.vocab


In [46]:
!git pull

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 4 (delta 2), reused 4 (delta 2), pack-reused 0 (from 0)
Unpacking objects: 100% (4/4), 1.11 KiB | 56.00 KiB/s, done.
From https://github.com/nakaikento/mt-ja-ko
   25e03b3..a5eab12  main       -> origin/main
Updating 25e03b3..a5eab12
Fast-forward
 training/generate_teacher_data.py | 43 +++++++++++++++++++++++----------------
 1 file changed, 26 insertions(+), 17 deletions(-)


In [49]:
rm data/teacher/train.ko

In [50]:
!ls data/teacher/train.ko

ls: cannot access 'data/teacher/train.ko': No such file or directory


In [54]:
def clean_and_sync():
    ja_in = "data/raw/OpenSubtitles.ja-ko.ja"
    ko_in = "data/teacher/train.ko"
    
    ja_out = "data/clean/train.ja"
    ko_out = "data/clean/train.ko"
    
    os.makedirs("data/clean", exist_ok=True)

    with open(ja_in, 'r', encoding='utf-8') as f_ja, \
         open(ko_in, 'r', encoding='utf-8') as f_ko, \
         open(ja_out, 'w', encoding='utf-8') as o_ja, \
         open(ko_out, 'w', encoding='utf-8') as o_ko:
        
        for ja_line, ko_line in zip(f_ja, f_ko):
            ja_line = ja_line.strip()
            ko_line = ko_line.strip()
            
            # 失敗行、空行、またはエラー文字列が含まれる場合はスキップ
            if "FAILED_TRANSLATION_CLEANED" in ko_line or not ko_line:
                continue
            
            o_ja.write(ja_line + "\n")
            o_ko.write(ko_line + "\n")

    print(f"✨ クリーニング完了！")
    print(f"📁 保存先: data/clean/train.ja, train.ko")

if __name__ == "__main__":
    clean_and_sync()

✨ クリーニング完了！
📁 保存先: data/clean/train.ja, train.ko


In [57]:
import random

def verify_parallel_data(ja_path, ko_path, num_samples=10):
    with open(ja_path, 'r', encoding='utf-8') as f_ja:
        ja_lines = [line.strip() for line in f_ja]
    with open(ko_path, 'r', encoding='utf-8') as f_ko:
        ko_lines = [line.strip() for line in f_ko]

    total = len(ja_lines)
    print("="*50)
    print(f"📊 データセット統計")
    print(f"  総ペア数: {total:,} 行")
    print("="*50)

    # ランダムにサンプルを抽出
    indices = random.sample(range(total), num_samples)
    
    print(f"\n🔍 ランダムサンプルチェック ({num_samples}件)")
    print("-" * 50)
    for idx in sorted(indices):
        ja = ja_lines[idx]
        ko = ko_lines[idx]
        
        # 簡易的な品質指標
        # 日本語と韓国語の長さの比率（極端に違う場合は注意）
        ratio = len(ko) / len(ja) if len(ja) > 0 else 0
        status = "✅" if 0.3 < ratio < 3.0 else "⚠️"

        print(f"[{idx+1}行目] {status}")
        print(f"  日: {ja}")
        print(f"  韓: {ko}")
        print(f"  (長さ比: {ratio:.2f})")
        print("-" * 30)

    # 最も長い文のチェック（バグ発見用）
    print("\n超長文チェック (異常な繰り返しがないか)")
    max_idx = max(range(total), key=lambda i: len(ja_lines[i]))
    print(f"[{max_idx+1}行目]")
    print(f"  日: {ja_lines[max_idx][:100]}...")
    print(f"  韓: {ko_lines[max_idx][:100]}...")

if __name__ == "__main__":
    verify_parallel_data("data/clean/train.ja", "data/clean/train.ko")

📊 データセット統計
  総ペア数: 605,964 行

🔍 ランダムサンプルチェック (10件)
--------------------------------------------------
[10353行目] ✅
  日: あんたが私をこんなにした
  韓: 넌 나를 이렇게 만들었지
  (長さ比: 1.08)
------------------------------
[18043行目] ✅
  日: それで何がわかったと思う？
  韓: 그래서 뭘 알았다고 생각하나?
  (長さ比: 1.23)
------------------------------
[228716行目] ✅
  日: コヨーテでいいのか？ - そうだ　ハンク
  韓: 코요테는 어때?
  (長さ比: 0.40)
------------------------------
[249598行目] ✅
  日: エイリアンは居ないわ
  韓: 에일리안은 여기 없어
  (長さ比: 1.10)
------------------------------
[260208行目] ✅
  日: - 誰かが支払わないとな
  韓: 누가 돈을 안 내면
  (長さ比: 0.83)
------------------------------
[316574行目] ✅
  日: やるぞ
  韓: 어서!
  (長さ比: 1.00)
------------------------------
[392388行目] ✅
  日: ええ ここで何をやっているんだ？ 聞かないと
  韓: 여기서 뭐하는 거야?
  (長さ比: 0.50)
------------------------------
[485977行目] ✅
  日: そんな事はない
  韓: 그런건 없지
  (長さ比: 0.86)
------------------------------
[530050行目] ✅
  日: やあ　どうも レックスです...
  韓: 안녕하세요, 렉스...
  (長さ比: 0.75)
------------------------------
[589965行目] ✅
  日: 彼は2番目のモジュールを作ったので 惑星全体を制御できます
  韓: 그는 두 번째 모듈을 

In [62]:
# ノートブックのセルで実行
import shutil

# NLLBモデルのキャッシュディレクトリ（RunPodの標準的なパス）
cache_dir = "/workspace/huggingface_cache/models--facebook--nllb-200-3.3b"

if os.path.exists(cache_dir):
    shutil.rmtree(cache_dir)
    print(f"✅ {cache_dir} を削除しました。空き容量が大幅に増えたはずです。")
else:
    print("❌ 指定のパスにキャッシュが見つかりませんでした。")

✅ /workspace/huggingface_cache/xet を削除しました。空き容量が大幅に増えたはずです。


## 4. MarianMT学習

In [13]:
# 基本
# python training/train_ko_ja.py

# カスタム設定
!python training/train_ko_ja.py \
    --epochs 10 \
    --batch-size 96 \
    --learning-rate 3e-4 \
    --output-dir models/ko-ja-v1

# # # チェックポイントから再開
# python training/train_ko_ja.py \
#     --resume models/ko-ja-v1/checkpoint-29000 \
#     --batch-size 96 \
#     --learning-rate 3e-4 \
#     --epochs 10 \
#     --output-dir models/ko-ja-v2

MarianMT 直接学習（韓国語→日本語）

デバイス: cuda
GPU: NVIDIA GeForce RTX 4090
VRAM: 25.4 GB

トークナイザー: data/tokenized/spm.model
語彙サイズ: 32,000

データ: data/splits
データ読み込み完了（韓国語→日本語）:
  Train: 1,025,749
  Val:   5,000
  Test:  5,000

トークナイズ中...
Tokenizing: 100%|█████████████████| 5000/5000 [00:00<00:00, 80724.58 examples/s]

モデル作成...
モデルパラメータ数: 61,046,784 (61.0M)

学習開始
  方向: 韓国語 → 日本語
  エポック数: 10
  バッチサイズ: 96 x 4 = 384
  学習率: 0.0003
  出力先: models/ko-ja-v1

{'loss': '36.41', 'grad_norm': '7.852', 'learning_rate': '2.97e-05', 'epoch': '0.03744'}
{'loss': '30.45', 'grad_norm': '2.857', 'learning_rate': '5.97e-05', 'epoch': '0.07487'}
{'loss': '28.1', 'grad_norm': '2.967', 'learning_rate': '8.97e-05', 'epoch': '0.1123'}
{'loss': '27.72', 'grad_norm': '3.668', 'learning_rate': '0.0001197', 'epoch': '0.1497'}
{'loss': '27.47', 'grad_norm': '7.397', 'learning_rate': '0.0001497', 'epoch': '0.1872'}
{'loss': '26.94', 'grad_norm': '3.251', 'learning_rate': '0.0001797', 'epoch': '0.2246'}
{'loss': '26.49', 'grad_no

In [15]:
# チェックポイントから再開
!python training/train_ko_ja.py \
    --resume models/ko-ja-v1/checkpoint-23000 \
    --batch-size 96 \
    --learning-rate 3e-4 \
    --epochs 10 \
    --output-dir models/ko-ja-v2

MarianMT 直接学習（韓国語→日本語）

デバイス: cuda
GPU: NVIDIA GeForce RTX 4090
VRAM: 25.4 GB

トークナイザー: data/tokenized/spm.model
語彙サイズ: 32,000

データ: data/splits
データ読み込み完了（韓国語→日本語）:
  Train: 1,025,749
  Val:   5,000
  Test:  5,000

トークナイズ中...
Tokenizing: 100%|█████████████████| 5000/5000 [00:00<00:00, 80091.05 examples/s]

モデル作成...
モデルパラメータ数: 61,046,784 (61.0M)

学習開始
  方向: 韓国語 → 日本語
  エポック数: 10
  バッチサイズ: 96 x 4 = 384
  学習率: 0.0003
  出力先: models/ko-ja-v2

There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.encoder.embed_positions.weight', 'model.decoder.embed_tokens.weight', 'model.decoder.embed_positions.weight', 'lm_head.weight'].
{'loss': '15.24', 'grad_norm': '4.8', 'learning_rate': '1.449e-05', 'epoch': '8.645'}
{'loss': '15.26', 'grad_norm': '4.481', 'learning_rate': '1.371e-05', 'epoch': '8.683'}
{'loss': '15.23', 'grad_norm': '4.542', 'learning_rate': '1.296e-05', 'epoch': '8.72'}
{'loss': '15.27', 'grad_norm': '4.702', 'learning_rate': '1.222e-05

## 5. モデル評価

In [16]:
import torch
from pathlib import Path
import sys
sys.path.append('/workspace/mt-ja-ko/mt-ja-ko')

from transformers import MarianMTModel
from training.train import SPMTokenizer

# モデルとトークナイザーをロード
model_path = "/workspace/mt-ja-ko/mt-ja-ko/models/ko-ja-v2"
model = MarianMTModel.from_pretrained(model_path)
tokenizer = SPMTokenizer(f"{model_path}/spm.model")

# GPU使用
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
model.eval()

def translate(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=128, num_beams=4)
    
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# テスト翻訳
test_sentences = [
    # ドラマ定番
    "날 믿어.",
    "거짓말하지 마.",
    "한 번만 더 기회를 주세요.",
    "널 좋아해.",
    "가지 마!",
    
    # 感情的なシーン
    "왜 그런 말을 해?",
    "다 내 잘못이야.",
    "네가 없으면 살 수 없어.",
    "용서해줘.",
    "다시는 보고 싶지 않아.",
    
    # アクション系
    "도망쳐!",
    "뒤에 누가 있어!",
    "시간이 없어. 빨리!",
    "나한테 맡겨.",
    "절대 포기하지 않을 거야.",
    
    # 日常ドラマ
    "오늘 야근이야?",
    "선배, 한잔하러 갈래요?",
    "남자친구랑 헤어졌어.",
    "결혼해 주세요.",
    "엄마, 미안해요.",
]

for ko in test_sentences:
    ja = translate(ko)
    print(f"🇰🇷 {ko}")
    print(f"🇯🇵 {ja}")
    print()

Loading weights:   0%|          | 0/254 [00:00<?, ?it/s]

🇰🇷 날 믿어.
🇯🇵 私を信じて

🇰🇷 거짓말하지 마.
🇯🇵 嘘をつくな

🇰🇷 한 번만 더 기회를 주세요.
🇯🇵 もう一回チャンスをくれ

🇰🇷 널 좋아해.
🇯🇵 あなたが好きなの。

🇰🇷 가지 마!
🇯🇵 行かないで!

🇰🇷 왜 그런 말을 해?
🇯🇵 なぜそんなことを?

🇰🇷 다 내 잘못이야.
🇯🇵 私のせいよ

🇰🇷 네가 없으면 살 수 없어.
🇯🇵 君なしでは生き残れない

🇰🇷 용서해줘.
🇯🇵 許してくれ

🇰🇷 다시는 보고 싶지 않아.
🇯🇵 二度と会いたくない

🇰🇷 도망쳐!
🇯🇵 逃げろ!

🇰🇷 뒤에 누가 있어!
🇯🇵 後ろに誰かいる!

🇰🇷 시간이 없어. 빨리!
🇯🇵 時間がない 早く!

🇰🇷 나한테 맡겨.
🇯🇵 私に任せて

🇰🇷 절대 포기하지 않을 거야.
🇯🇵 決して諦めない

🇰🇷 오늘 야근이야?
🇯🇵 今日は暑いのか?

🇰🇷 선배, 한잔하러 갈래요?
🇯🇵 飲み物でも飲むか?

🇰🇷 남자친구랑 헤어졌어.
🇯🇵 彼氏と別れた

🇰🇷 결혼해 주세요.
🇯🇵 結婚して

🇰🇷 엄마, 미안해요.
🇯🇵 ママ、ごめん。



## 6. ONNX変換（Android用）

In [1]:
# 依存関係インストール
!pip install -q optimum[onnxruntime]


[notice] A new release of pip is available: 24.2 -> 26.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [19]:
!pip install --upgrade transformers optimum

  Using cached transformers-5.1.0-py3-none-any.whl.metadata (31 kB)
Using cached transformers-5.1.0-py3-none-any.whl (10.3 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 553.3/553.3 kB 82.5 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface_hub 0.36.2
    Uninstalling huggingface_hub-0.36.2:
      Successfully uninstalled huggingface_hub-0.36.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.6
    Uninstalling transformers-4.57.6:
      Successfully uninstalled transformers-4.57.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
optimum-onnx 0.1.0 requires transformers<4.58.0,>=4.36, but you have transformers 5.1.0 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 26.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
# ONNX変換（韓国語→日本語）
from optimum.onnxruntime import ORTModelForSeq2SeqLM
from pathlib import Path
import shutil
import os

# 絶対パスを使用
base_dir = "/workspace/mt-ja-ko/mt-ja-ko"
model_path = f"{base_dir}/models/ko-ja-v2"
output_path = f"{base_dir}/models/ko-ja-onnx"

# パス確認
print(f"モデルパス: {model_path}")
print(f"存在確認: {os.path.exists(model_path)}")

# 変換（local_files_only=True でローカルから読み込み）
print("ONNX変換中...")
ort_model = ORTModelForSeq2SeqLM.from_pretrained(
    model_path,
    export=True,
    local_files_only=True
)

# 保存
ort_model.save_pretrained(output_path)

# トークナイザーもコピー
shutil.copy(f"{model_path}/spm.model", f"{output_path}/spm.model")

print(f"✅ 完了: {output_path}")

# サイズ確認
!ls -lh {output_path}/
!du -sh {output_path}/

`torch_dtype` is deprecated! Use `dtype` instead!


モデルパス: /workspace/mt-ja-ko/mt-ja-ko/models/ko-ja-v2
存在確認: True
ONNX変換中...


/usr/local/lib/python3.11/dist-packages/transformers/modeling_attn_mask_utils.py:196: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  inverted_mask = torch.tensor(1.0, dtype=dtype) - expanded_mask
/usr/local/lib/python3.11/dist-packages/transformers/integrations/sdpa_attention.py:81: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  is_causal = query.shape[2] > 1 and attention_mask is None and getattr(module, "is_causal", True)
/usr/local/lib/python3.11/dist-packages/transformers/models/marian/modeling_marian.py:642: Tra

✅ 完了: /workspace/mt-ja-ko/mt-ja-ko/models/ko-ja-onnx
total 570M
-rw-rw-rw- 1 root root  876 Feb  8 00:45 config.json
-rw-rw-rw- 1 root root 223M Feb  8 00:45 decoder_model.onnx
-rw-rw-rw- 1 root root 211M Feb  8 00:45 decoder_with_past_model.onnx
-rw-rw-rw- 1 root root 136M Feb  8 00:45 encoder_model.onnx
-rw-rw-rw- 1 root root  112 Feb  8 00:45 generation_config.json
-rw-rw-rw- 1 root root 807K Feb  8 00:45 spm.model
572M	/workspace/mt-ja-ko/mt-ja-ko/models/ko-ja-onnx/


In [103]:
# 変換後のテスト
from optimum.onnxruntime import ORTModelForSeq2SeqLM
import sys
sys.path.append('/workspace/mt-ja-ko/mt-ja-ko')
from training.train import SPMTokenizer

# ONNXモデルをロード
ort_model = ORTModelForSeq2SeqLM.from_pretrained(output_path)
tokenizer = SPMTokenizer(f"{output_path}/spm.model")

def translate_onnx(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True)
    outputs = ort_model.generate(**inputs, max_length=128, num_beams=4)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# テスト
test = [
    "こんにちは",
    "逃げろ！",
    "君のことが好きだ。",
]

for ja in test:
    ko = translate_onnx(ja)
    print(f"🇯🇵 {ja} → 🇰🇷 {ko}")

Could not find any ONNX files with standard file name decoder_model_merged.onnx, files found: [PosixPath('decoder_with_past_model.onnx'), PosixPath('decoder_model.onnx'), PosixPath('encoder_model.onnx')]. Please make sure to pass a `file_name` and/or `subfolder` argument to `from_pretrained` when loading an ONNX file with non-standard file names.


🇯🇵 こんにちは → 🇰🇷 안녕하세요
🇯🇵 逃げろ！ → 🇰🇷 도망쳐!
🇯🇵 君のことが好きだ。 → 🇰🇷 너에 대해 좋아해


In [4]:
# 量子化して軽量化（韓国語→日本語）
from optimum.onnxruntime import ORTQuantizer
from optimum.onnxruntime.configuration import AutoQuantizationConfig
import shutil
from pathlib import Path

base_dir = "/workspace/mt-ja-ko/mt-ja-ko"
output_path = f"{base_dir}/models/ko-ja-onnx"
quantized_path = f"{base_dir}/models/ko-ja-onnx-int8"

Path(quantized_path).mkdir(exist_ok=True)

# 量子化設定
qconfig = AutoQuantizationConfig.avx512_vnni(is_static=False)

# 全ONNXファイルを量子化
onnx_files = [
    "encoder_model.onnx",
    "decoder_model.onnx", 
    "decoder_with_past_model.onnx"
]

for onnx_file in onnx_files:
    print(f"量子化中: {onnx_file}")
    quantizer = ORTQuantizer.from_pretrained(output_path, file_name=onnx_file)
    quantizer.quantize(save_dir=quantized_path, quantization_config=qconfig)

# 設定ファイルとトークナイザーをコピー
shutil.copy(f"{output_path}/config.json", quantized_path)
shutil.copy(f"{output_path}/generation_config.json", quantized_path)
shutil.copy(f"{output_path}/spm.model", quantized_path)

print("\n✅ 量子化完了!")

# サイズ確認
!ls -lh {quantized_path}/
!du -sh {quantized_path}/

量子化中: encoder_model.onnx
量子化中: decoder_model.onnx
量子化中: decoder_with_past_model.onnx

✅ 量子化完了!
total 146M
-rw-rw-rw- 1 root root  876 Feb  8 00:47 config.json
-rw-rw-rw- 1 root root  57M Feb  8 00:47 decoder_model_quantized.onnx
-rw-rw-rw- 1 root root  54M Feb  8 00:47 decoder_with_past_model_quantized.onnx
-rw-rw-rw- 1 root root  35M Feb  8 00:47 encoder_model_quantized.onnx
-rw-rw-rw- 1 root root  112 Feb  8 00:47 generation_config.json
-rw-rw-rw- 1 root root  762 Feb  8 00:47 ort_config.json
-rw-rw-rw- 1 root root 807K Feb  8 00:47 spm.model
148M	/workspace/mt-ja-ko/mt-ja-ko/models/ko-ja-onnx-int8/


In [10]:
# !pip install datasets
# !pip install sentencepiece
!pip install evaluate


[notice] A new release of pip is available: 24.2 -> 26.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [11]:
from optimum.onnxruntime import ORTModelForSeq2SeqLM
import sys
sys.path.append('/workspace/mt-ja-ko/mt-ja-ko')
from training.train import SPMTokenizer

# 量子化モデルをロード
quantized_path = "/workspace/mt-ja-ko/mt-ja-ko/models/ko-ja-onnx-int8"
ort_model = ORTModelForSeq2SeqLM.from_pretrained(quantized_path)
tokenizer = SPMTokenizer(f"{quantized_path}/spm.model")

def translate_q(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True)
    outputs = ort_model.generate(**inputs, max_length=128, num_beams=4)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# 量子化前後で品質比較
test = [
    # ドラマ定番
    "날 믿어.",
    "거짓말하지 마.",
    "한 번만 더 기회를 주세요.",
    "널 좋아해.",
    "가지 마!",
    
    # 感情的なシーン
    "왜 그런 말을 해?",
    "다 내 잘못이야.",
    "네가 없으면 살 수 없어.",
    "용서해줘.",
    "다시는 보고 싶지 않아.",
    
    # アクション系
    "도망쳐!",
    "뒤에 누가 있어!",
    "시간이 없어. 빨리!",
    "나한테 맡겨.",
    "절대 포기하지 않을 거야.",
    
    # 日常ドラマ
    "오늘 야근이야?",
    "선배, 한잔하러 갈래요?",
    "남자친구랑 헤어졌어.",
    "결혼해 주세요.",
    "엄마, 미안해요.",
]

print("【量子化モデル テスト（韓国語→日本語）】\n")
for ko in test:
    ja = translate_q(ko)
    print(f"🇰🇷 {ko}")
    print(f"🇯🇵 {ja}")
    print()

Could not find any ONNX files with standard file name encoder_model.onnx, files found: [PosixPath('decoder_with_past_model_quantized.onnx'), PosixPath('decoder_model_quantized.onnx'), PosixPath('encoder_model_quantized.onnx')]. Please make sure to pass a `file_name` and/or `subfolder` argument to `from_pretrained` when loading an ONNX file with non-standard file names.
Could not find any ONNX files with standard file name decoder_model_merged.onnx, files found: [PosixPath('decoder_with_past_model_quantized.onnx'), PosixPath('decoder_model_quantized.onnx'), PosixPath('encoder_model_quantized.onnx')]. Please make sure to pass a `file_name` and/or `subfolder` argument to `from_pretrained` when loading an ONNX file with non-standard file names.
Could not find any ONNX files with standard file name decoder_model.onnx, files found: [PosixPath('decoder_with_past_model_quantized.onnx'), PosixPath('decoder_model_quantized.onnx'), PosixPath('encoder_model_quantized.onnx')]. Please make sure to p

【量子化モデル テスト（韓国語→日本語）】

🇰🇷 날 믿어.
🇯🇵 私を信じて

🇰🇷 거짓말하지 마.
🇯🇵 嘘をつくな

🇰🇷 한 번만 더 기회를 주세요.
🇯🇵 もう一回チャンスをくれ

🇰🇷 널 좋아해.
🇯🇵 あなたが好きなの。

🇰🇷 가지 마!
🇯🇵 行かないで!

🇰🇷 왜 그런 말을 해?
🇯🇵 なぜそんなことを?

🇰🇷 다 내 잘못이야.
🇯🇵 私のせいよ

🇰🇷 네가 없으면 살 수 없어.
🇯🇵 君なしでは生き残れない

🇰🇷 용서해줘.
🇯🇵 許してくれ

🇰🇷 다시는 보고 싶지 않아.
🇯🇵 二度と会いたくない

🇰🇷 도망쳐!
🇯🇵 逃げろ!

🇰🇷 뒤에 누가 있어!
🇯🇵 後ろに誰かいる!

🇰🇷 시간이 없어. 빨리!
🇯🇵 時間がない 早く!

🇰🇷 나한테 맡겨.
🇯🇵 私に任せて

🇰🇷 절대 포기하지 않을 거야.
🇯🇵 決して諦めない

🇰🇷 오늘 야근이야?
🇯🇵 今日は暑いのか?

🇰🇷 선배, 한잔하러 갈래요?
🇯🇵 飲み物は?

🇰🇷 남자친구랑 헤어졌어.
🇯🇵 彼氏と別れた

🇰🇷 결혼해 주세요.
🇯🇵 結婚して

🇰🇷 엄마, 미안해요.
🇯🇵 ママ、ごめんなさい。



## 7. ダウンロード

学習済みモデルは `workspace/models/` に保存されています。

必須ファイル:
- encoder_model_quantized.onnx (35MB)
- decoder_model_quantized.onnx (57MB)
- spm.model (807KB)

高速推論用（推奨）:

- decoder_with_past_model_quantized.onnx (54MB)

設定ファイル:

- config.json
- generation_config.json